# Tutorial 6: Kinetic Analysis for Time Resolved, Temperature-Jump SAXS Data Analysis

**Package Information:**<br>
Currently the [tr_tjump_saxs](https://gitlab.oit.duke.edu/tr_t-jump_saxs/y22-23/-/tree/main?ref_type=heads "tr_tjump_saxs") package only works through the Python3 command line. The full dependencies can be found on the [Henderson GitLab Page](https://gitlab.oit.duke.edu/tr_t-jump_saxs/y22-23/-/tree/main?ref_type=heads "tr_tjump_saxs") and the environment can be cloned from the [environment.yml file](https://gitlab.oit.duke.edu/tr_t-jump_saxs/y22-23/-/blob/main/environment.yml?ref_type=heads "environment.yml file"). The data analysis can be executed from an interactive Python command line such as [iPython](https://www.python.org/) or [Jupyter](https://jupyter.org/) or the code can be written in a script to run in a non-interactive mode. The preferred usage is in Jupyter Lab as this is the environment the package was developed in. Jupyter also provides a file where all code, output of code, and notes can be contained in a single file and serves as a record of the data analysis performed, the code used to conduct the data analysis, and the output of the analysis. 

**Tutorial Information:**<br>
This set of tutorial notebooks will cover how to use the `tr_tjump_saxs` package to analyze TR, T-Jump SAXS data and the <a href="https://www.science.org/doi/10.1126/sciadv.adj0396">workflow used to study HIV-1 Envelope glycoprotein dynamics. </a> This package contains multiple modules, each containing a set of functions to accomplish a specific subtask of the TR, T-Jump SAXS data analysis workflow. Many of the functions are modular and some can be helpful for analyzing static SAXS and other data sets as well. 

**Package Modules:**<br>
> 1. `file_handling`<br>
> 2. `saxs_processing`<br>
> 3. `saxs_qc`<br>
> 4. `saxs_kinetics`<br>
> 5. `saxs_modeling`<br>

**Developer:** 
Ashley L. Bennett | PhD <br>
> GitHub: [@ScientistAsh](https://github.com/ScientistAsh "ScientistAsh GitHub") <br>
> GitLab: [@ab875](https://gitlab.oit.duke.edu/ab875 "ab875") <br>

**Updated:** 26 February 2024

# Tutorial 6 Introduction
In this Tutorial 6 notebook, I introduce the `saxs_kinetics` module from the `tr_tjump_saxs` package. The `saxs_kinetics` module provides functions that will perform kinetic analysis on a set of TR, T-Jump SAXS difference curves. If you find any issues with this tutorial, please create an issue on the repository GitLab page ([tr_tjump_saxs issues](https://gitlab.oit.duke.edu/tr_t-jump_saxs/y22-23/-/issues "tr_tjump_saxs Issues")).

## Module functions:
> `saxs_auc()` calculates the area under the curve according to Simpson's Rule for given curves. <br>
> `svd_kinetics()` performs an SVD analysis to identify individual components. <br>
> `auc_fit()` fits either a single or double exponential decay function to input AUC data. <br>
> `svd_fit()` fits either a single or double exponential decay function to input SVD right vectors. <br>

## Tutorial Files:

### Data Files
The original data used in this analysis is deposited on the [SASBDB](https://www.sasbdb.org/) with accession numbers:
> **Static Data:** <br>
    - *CH505 Temperature Sereies*: SASDT29, SASDT39, SASDT49, SASDT59 <br>
    - *CH848 Temperature Series*: SASDTH9, SASDTJ9, SASDTK9, SASDTL9 <br>
<br>
> **T-Jump Data:** <br>
    - *CH505 T-Jump Data*: SASDT69, SASDT79, SASDT89, SASDT99, SASDTA9, SASDTB9, SASDTC9, SASDTD9, SASDTE9, SASDTF9, SASDTG9 <br>
     - *CH848 T-Jump Data*: SASDTM9, SASDTN9, SASDTP9, SASDTQ9, SASDTR9, SASDTS9, SASDTT9, SASDTU9, SASDTV9, SASDTW9 <br>
<br>
> **Static Env SOSIP Panel:** SASDTZ9, SASDU22, SASDU32, SASDU42, SASDTX9, SASDTY9 <br>

Additional MD data associated with the paper can be found on [Zenodo](https://zenodo.org/records/10451687).

### Output Files
Example output is included in the [OUTPUT](https://gitlab.oit.duke.edu/tr_t-jump_saxs/y22-23/-/tree/main/TUTORIALS/OUTPUT?ref_type=heads) subdirectory in the [TUTORIALS](https://gitlab.oit.duke.edu/tr_t-jump_saxs/y22-23/-/tree/main/TUTORIALS?ref_type=heads) directory.  

# How to Use Jupyter Notebooks
You can execute the code directly in this notebook or create your own notebook and copy the code there.


<div class="alert alert-block alert-info">
    
    <b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; Tips</b><br>
    
    <b>1.</b> To run the currently highlighted cell, hit the <code>shift</code> and <code>enter</code> keys at the same time.<br>
    <b>2</b>. To get help with a specific function, place the cursor in the functions brackets and hit the <code>shift</code> and <code>tab</code> keys at the same time.

</div>

<div class="alert alert-block alert-info" style="background-color: white; border: 2px solid; padding: 10px">
    <b><i class="fa fa-star" aria-hidden="true"></i>&nbsp; In the Literature</b><br>
    
    Our <a href="https://www.science.org/doi/10.1126/sciadv.adj0396">recent paper </a> in Science Advances provides an example of the type of data, the analysis procedure, and example output for this type of data analysis.  <br> 
    
    <p style="text-align:center">
    
</div>

# Import Modules

The first step is to import the necessary python packages. The dependecies will automatically be imported with the package import.

In [ ]:
# import sys to allow python to use the file browser to find files
import sys

# append the path for the tr_tjump_saxs_analysis package to the PYTHONPATH
sys.path.append(r'../')

# import CH505TF_SAXS analysis dependent packages and custom functions
from file_handling import *
from saxs_processing import *
from saxs_qc import *
from saxs_kinetics import *

<div class="alert alert-block alert-info">
    <b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; Tips</b><br>
    Be sure that the path for the <code>tr_tjump_saxs</code> package appended to the <code>PYTHONPATH</code> matches the path to the repository on your machine.
    </div>

# TR, T-Jump Kinetic Analysis Overview
Extracting kinetics from TR, T-Jump SAXS data is a multi-step process. In general, this involves fitting models to the extracted components. However, there are many different options for model selection and how to identify components. The best approach will be dictated by the data set and the specific question of interest. This notebook tutorial will cover the modeling pipeline we used to study [HIV-1 Envelope Glycoprotein structural dynamics](https://www.science.org/doi/10.1126/sciadv.adj0396).

The overview of our SAXS kinetics pipeline is shown in [Figure 1](#Figure-1). The first step is to **identify the individual components** contributing to the SAXS difference signal. SVD is the typical method used to extract these components as shown below. Once the number of components is identified by SVD, **constructing a [REGALS](https://github.com/ando-lab/regals) model** yields more physically meaningful components and calculates pair-distance distribution difference and component contributions. The REGALS model should be built using the same number of components indicated by the SVD with d$_{max}$ values determined from the either structural model dimensions or the R$_g$ determined from the Guinier analysis.

Once the individual components are identified, the next step is to **fit the components** to a model. The most straight-forward way is to fit the model to the right vectors output from the REGALS (and/or SVD) analysis. Alternatively, the area under each TR, T-Jump SAXS difference curve can be calculated and this AUC plot vs. time delay can be fit to a model. Both methods yield results within error of each other. Importantly, *the number of components used in the fits for SVD, REGALS, and/or AUC data should de determined from the number of components indicated by the SVD analysis*. For example, if the SVD analysis indicates 2 components, then a double exponential decay should be used for fitting rather than a single exponential decay. You should also use the fewest number of components when fitting the model that is allowable (i.e. unless the data indicates otherwise, the simpler model should be preferred). 

Currently, the `saxs_kinetics` module only includes functions for running SVD and AUC analysis as well as fitting the output of the SVD and AUC analyses. REGALS is developed by another group and can be cloned from the [REGALS GitHub Page](https://github.com/ando-lab/regals). Once REGALS is downloaded it can be imported as Python module and the functions can be used in a manner similar to the usage for this package. 

<a id='Figure-1'></a>

<img src="saxs_kinetics.png" alt="SAXS modeling workflow" width="800" align="center">

**Figure 1:** Workflow to extract kinetics from TR, T-Jump SAXS data. Note that in our final publication fit the SVD right vectors instead of AUC as shown here. Either method is acceptable and yields the same results (within error). 

# Extracting Components with SVD Analysis

## Function Overview
In the `saxs_kinetics` module, the `svd_kinetics()` function will run the SVD analysis on a given set of input TR, T-Jump SAXS curves. The average, buffer subtracted curves, calculated using the methods described in the previous tutorials, should be used as input for the SVD analysis. The range of time delays used should cover the time scale range of interest. This function will save a scree plot along with output plots of both the first two left vectors and the first two right vectors will be made, with the left and right vectors in separate plots. CSV files of the right and left vectors are also saved.

## Input Parameters
There are four input parameters for the `svd_kinetics()` function, only one of which is required: 
> 1. `flist` is the only required parameter and indicates a list of files containing the average, buffer-subtracted TR, T-Jump SAXS difference curve set for analysis. <br>
> 2. `times` is a list of int indicating the time delays associated with the given input curves. The default values are [10, 50, 100, 500, and 1000]. <br>
> 3. `time_unit` is a str indicating the units for the time delays. The default value is us (for microseconds). 
> 4. `outdir` is a str indicating the name of directory to store output plots to.  If set to None, then no outfile will be saved. Default value is None. Note that the saved plots have the axis labels hard-coded. Future version of this code will remove the hard-coding of labels, but until then these labels cannot be changed by the user using the input parameters. <br>

## Returned Values
> 1. A numpy array containing the left vectors describing the average components of the input data. The dimensions have the same size as those of the input. <br>
> 2. A numpy array containing the singular values describing the significance of each component to contributing to the total signal, sorted in descending order. The dimensions have the same size as those of the input.
> 3. A numpy array containing the right vectors describing the contribution each left vector makes at each time point. The dimensions have the same size as those of the input. 

## Raised Errors
There are no custom errors raised by this function. If any errors are raised, refer to the documentation for the function indicated in the TraceBack. 

## Example 1: Basic Usage
### Step 1: Define Data Set Variables

In [ ]:
# define times 
times = [3, 5, 10, 50, 100, 500, 1000]

# define files
files = ['../../../TR_T-jump_SAXS_July2022/ANALYSIS/PROTEIN/TJUMP/20hz_set02/BUFFER_SUB/3us_buff_sub_SEMfix.csv',
        '../../../TR_T-jump_SAXS_July2022/ANALYSIS/PROTEIN/TJUMP/20hz_set02/BUFFER_SUB/5us_buff_sub_SEMfix.csv',
        '../../../TR_T-jump_SAXS_July2022/ANALYSIS/PROTEIN/TJUMP/20hz_set01/BUFFER_SUB/10us_buff_sub_SEMfix.csv',
        '../../../TR_T-jump_SAXS_July2022/ANALYSIS/PROTEIN/TJUMP/20hz_set01/BUFFER_SUB/50us_buff_sub_SEMfix.csv',
        '../../../TR_T-jump_SAXS_July2022/ANALYSIS/PROTEIN/TJUMP/20hz_set01/BUFFER_SUB/100us_buff_sub_SEMfix.csv',
        '../../../TR_T-jump_SAXS_July2022/ANALYSIS/PROTEIN/TJUMP/20hz_set01/BUFFER_SUB/500us_buff_sub_SEMfix.csv',
        '../../../TR_T-jump_SAXS_July2022/ANALYSIS/PROTEIN/TJUMP/20hz_set01/BUFFER_SUB/1ms_buff_sub_SEMfix.csv']

### Step 2: Run SVD Analysis

In [ ]:
u,s,v = svd_kinectics(flist=files, times=times, time_unit='us', 
                      outdir='./OUTPUT/TUTORIAL6/SVD/')

As you can see in the scree plot 2 components are required to fit this data. Remember that Python arrays are 0-indexed. The scree plot shows that the components above 1 do not contribute substantially to the signal. However, because of the 0-indexing in the arrays, the indication of 1 component in the scree plot actually indicates 2 components required to model the data. 

The left vector plots describe the average components for the first 2 left vectors. The right vector plot shows the first two right vectors and the contribution each component makes to the individual input difference curves. 

The right and left vectors will be saved as a CSV file in the indicated directory. The singular values are not automatically saved by the function. 

In [ ]:
# save singular value
np.savetxt('./OUTPUT/TUTORIAL6/SVD/singular_values.csv', s)

# Extracting Kinetics by Fitting SVD Right Vectors

## Function Overview
One way to extract the kinetics from TR, T-Jump SAXS difference curves is to fit the SVD extracted right vectors to an approprote model. The appropriate model to use will be determined by the specific question at hand and the number of parameters to fit in the model is determined by the number of components indicated by the SVD analysis. The `svd_fit()` function will fit either a single exponential decay or double exponential decay model to a given right vector. The form of the models used are:
> Single exponential: A * np.exp(-x/tau) + B <br>
> Double exponential: A1 * np.exp(-x/tau1) + A2 * np.exp(-x/tau2) + B <br>

## Input Parameters
There are many input parameters, but only the first two are required.
> 1. `file` is a string containing the file name, with full path, containing the data to be fit.<br> 
> 2. `x` is a numpy array containing the X-data. <br>
> 3. `delim` is an optional string indicating the delimitter used in file to load. Default value is comma. <br>
> 4. `row` is optional and is an integer indicating which row contains the right vector to fit. Rows are 0 indexed. Default value is 0, which corresponds to the first right vector. <br>
> 5. `func` is an optional string parameter that indicates what type of function to fit to data. Accepts either 'double' for double exponential decay fit or 'single' for single exponential decay fit. <br>
> 6. `initial_guess` is an optional array-like parameter for the initial guess for the fitted parameters. The default sets all the initial values to 1. <br>
> 7. `iterations` is an optional paramter. This integer indicates the number of iterations to run the scipy.optimize.curve_fit. Default value is 2000. <br>
> 8. `xlab` is an optional string parameter to use as the label of the x-axis in plots. The default value is 'Time Delay (us).' <br>
> 9. `ylab` is an optional string parameter to use as the label of the y-axis in plots. The default value is '% Contribution'. <br>
> 10. `xlogscale` is an optional boolean parameter indicating if a log scale should be used on the x axis. If set to False then the absolute values will be used. Default value is True. <br>
> 11. `ylogscale` is an optional boolean parameter indicating if a log scale should be used on the y-axis. If set to False then the absolute values will be used. Default value is False. <br>
> 12. `outdir` is an optional string parameter indicating the location to save the output plots and modeled curve. If set to None then no CSV or PNG plot files will be saved. Default value is None. <br>
> 13. `outfile` is an optional string parameter indicating the prefix to use for file names. CSV files containing the fitted curve will be saved with the '.csv' suffix and plot files will be saved with the '.png' suffix. Default value is 'fit'. <br>
> 14. `plot_title` is an optional string parameter that is used as the label to use for title of plot. Default value is 'CH505TF SAXS T-Jumps First Right Vector\n Exponential Fit'.<br>

## Returned Values
This function returns 3 values:
> 1. popt is a numpy array containing the values of funcotional parameters determined from exponential fit. <br>
> 2. conf_int is a numpy array describing the confidence intervals for the fitted parameters. <br>
> 3. model is a numpy array containing the model of fit. <br>

## Raised Errors
This function will raise a ValueError if something other than 'single' or 'double' is passed as input for the `func` parameter. If this error is rasied then the input needs to be changed to 'single' for a single exponential fit or 'double' for a double exponential fit. 

## Example 2: Basic Usage

In [ ]:
popt, conf_int, model = svd_fit(file='./OUTPUT/TUTORIAL6/SVD/right_vectors.csv', x=times, delim=',', row=0, 
        func='double', initial_guess=[10, 5, 20, 700.0, 15],iterations=2000, 
        xlab='Time Delay ($\mu$s; Log Scale)', ylab='% Contribution', xlogscale=True, 
        ylogscale=True, outdir='./OUTPUT/TUTORIAL6/SVD/', outfile='rv1_double_exp_decay_fit', 
        plot_title='CH505TF SAXS T-Jumps Second Right Vectors\nDouble Exponential Decay Fit')

You can see that the fit is not perfect. The initial guess could potentially be modified to get a better fit. For a thorough analysis you would want to fit all the right vectors that are indicated by SVD as a siginificant components. 

# Extracting Kinetics from AUC Analysis

## Function Overview
Calculating the AUC for each time delay and fitting this data to an appropriate equation is an alternative way to extract the TR, T-Jump SAXS data. Fitting AUC and SVD right vectors should yield similar results for the extracted kinetic values. The `saxs_auc()` function will calculate the AUC for a given SAXS scattering or difference curve. 

## Input Parameters
> 1. `flist`: a list of files, including full path,  containing curves to be loaded and averaged. When set to none, will load files with given prefix/suffix from the indicated data_dir. Default is None. Note that either flist or the directory storing files must be given. This is the only required parameter. <br> 
> 2. `times`: a list of times corresponding to the data to run AUC analysis. Will be used as the X-label values for plots. The default value is [1.5, 3, 5, 10, 50, 100, 300, 500, 1000]. <br>
> 3. `delim`: a string indicating the delimitter used in data files. Default is comma. <br>
>4. `mask`: is an integer indicating the number of rows to skip when loading files. Helpful when a mask was applied during data collection and low scattering vector values should not be analyzed or if metadata/headers are contained in the file. When set to 0 all rows are loaded. Default value is 0. <br>
> 5. `q_min`: is a float indicating the q value to begin integration at. If set to None, then will start at lowest q value in input data set. Default value is None. <br>
> 6. `q_max`: is a float indicating the q value to end integration at. If set to None, then will end at highest q value in input data set. Default value is None. <br>
> 7. `outdir`: is a string indicating the directory to save calculated area under the curve calculations and plots to. When set to None, no files will be saved. Default is None. <br>
> 8. `data_outfile`: is a string indicating the name of saved file storing the calculated average. <br>
> 9. `plot_outfile`: is a string indicating the name of saved file containing the plot. File will be saved as PNG in outdir. When set to None, no file will be saved. <br>
> 10. `xlab`: is a string used as a label for the X-axis in plot. Default value is 'Time Delay (us).'<br>
> 11. `plot_title`: is a string that is used as the label for title of plot. Default value is 'CH505TF SAXS T-Jumps Area Under the Curve'. <br>

## Returned Values
> 1. `auc`: a numpy array containing the time delay and calculated AUC values.  

## Raised Errors 
This function does not raise any custom errors. If errors are raised, refer to the documentation for the function indicated the TraceBack error. 

## Example 3: Basic Usage

### Step 1: Define Data Set Variables

In [ ]:
# define times 
times = [3, 5, 10, 50, 100, 500, 1000]

# define files
files = ['../../../TR_T-jump_SAXS_July2022/ANALYSIS/PROTEIN/TJUMP/20hz_set02/BUFFER_SUB/3us_buff_sub_SEMfix.csv',
        '../../../TR_T-jump_SAXS_July2022/ANALYSIS/PROTEIN/TJUMP/20hz_set02/BUFFER_SUB/5us_buff_sub_SEMfix.csv',
        '../../../TR_T-jump_SAXS_July2022/ANALYSIS/PROTEIN/TJUMP/20hz_set01/BUFFER_SUB/10us_buff_sub_SEMfix.csv',
        '../../../TR_T-jump_SAXS_July2022/ANALYSIS/PROTEIN/TJUMP/20hz_set01/BUFFER_SUB/50us_buff_sub_SEMfix.csv',
        '../../../TR_T-jump_SAXS_July2022/ANALYSIS/PROTEIN/TJUMP/20hz_set01/BUFFER_SUB/100us_buff_sub_SEMfix.csv',
        '../../../TR_T-jump_SAXS_July2022/ANALYSIS/PROTEIN/TJUMP/20hz_set01/BUFFER_SUB/500us_buff_sub_SEMfix.csv',
        '../../../TR_T-jump_SAXS_July2022/ANALYSIS/PROTEIN/TJUMP/20hz_set01/BUFFER_SUB/1ms_buff_sub_SEMfix.csv']

### Step 2: Calculate AUC

In [ ]:
auc = saxs_auc(flist=files, times=times, delim=',', mask=0, qmin=0.02, qmax=0.1, 
               outdir='./OUTPUT/TUTORIAL6/AUC/', 
               data_outfile='tutorial6_ex3_step2.csv', 
               plot_outfile='tutorial6_ex3_step2.png', xlab='Time Delay ($\\mu$s)',
               plot_title='CH505TF SAXS T-Jumps Area Under the Curve')

The `saxs_auc()` function only calculates and plots the area under the curve at each time delay. The plot connects the dots to make the trend easier to visualize but **IS NOT** a fitted model. To fit the AUC we just calculated we need to use the `auc_fit()` function. This `auc_fit()` function is similar to the `svd_fit()` function and has the same input and output. 

### Step 3: Fit AUC

In [ ]:
popt, conf_int, model = auc_fit(file='./OUTPUT/TUTORIAL6/AUC/tutorial6_ex3_step2.csv', x=times, 
                                columns=None, delim=',', skip=0, func='double', 
                                initial_guess=[10, 5, 20, 700.0, 15], iterations=2000, 
                                xlab='Time Delay ($\mu$s; Log Scale)', ylab='% Contribution', 
                                xlogscale=False, ylogscale=False, outdir='./OUTPUT/TUTORIAL6/AUC/', 
                                outfile='auc_double_exp_decay_fit', 
                                plot_title='CH505TF SAXS T-Jumps Second Right Vectors\nDouble Exponential Decay Fit')

# Extracting Kinetics Using REGALS
SVD analysis is great for identifying components of a given signal and reducing noise. However, the process of SVD can create physically unmeaningful rotations/translations of the the vectors. [REGALS](https://github.com/ando-lab/regals) is a method for running and SVD analysis that is constrained by parameters of the physical system in question, such as the maximum dimensions of the protein. REGALS is developed by another group and the installation and usage instructions can be found at the link noted above. Here I will demonstrate an example of how to run REGALS on this CH505TF dataset

## Step 1: Import Modules

In [ ]:
sys.path.append(r'../../src/regals/python/')
import nifty
import efa
import regals

## Step 2: Load Data

In [ ]:
set1 = np.loadtxt(fname='../../../TR_T-jump_SAXS_July2022/protein_20hz_set01_processed.csv',
                 skiprows=1, delimiter=',')
set2 = np.loadtxt(fname='../../../TR_T-jump_SAXS_July2022/protein_20hz_set02_processed.csv',
                 skiprows=1, delimiter=',')
set3 = np.loadtxt(fname='../../../TR_T-jump_SAXS_July2022/protein_5hz_set01_processed.csv',
                 skiprows=1, delimiter=',')

# Restructure data
set1_T = set1.transpose()
set2_T = set2.transpose()
set3_T = set3.transpose()

# Filter data
q = set1_T[0]

set1_data = set1_T[1::2]
set1_err = set1_T[2::2]

set2_data = set2_T[1::2]
set2_err = set2_T[2::2]

set3_data = set3_T[1::2]
set3_err = set3_T[2::2]

# combine data
data = np.vstack((set2_data, set1_data))
err = np.vstack((set2_err, set1_err))

<div class="alert alert-block alert-warning">
    
    <i class="fa fa-exclamation-triangle"></i>&nbsp; <b>Check your data structure</b><br>
    Your data may be stored differently and you may not need to restructure, filter, and/or combine data. Once your data is loaded make sure you understand your data structure and that the data has the appropriate structure before beginning any analysis. Once you load your data, you should plot it to be sure it has the right structure. After restructuring data, you should also plot to make sure the restructuring worked as expected. 
    </div>


## Step 3: Mask WAXS Data

In [ ]:
qmask = len(np.where(q <= 0.3)[0])
newq = q[:qmask]
imask = data[:, :qmask]
errmask = err[:, :qmask]

## Step 4: Transpose Data

In [ ]:
iT = np.transpose(imask)
print(iT.shape)
errT = np.transpose(errmask)
print(errT.shape)

## Step 5: Define Components

In [ ]:
x = [1.5, 3, 5, 10, 50, 100, 500, 1000]
x = np.log10(x) # we're going to seek smoothness on a logarithmic scale

#C1 = regals.component(
#    concentration_class('smooth', x, xmin = 0.1, xmax = 3.5, is_zero_at_xmin = True, is_zero_at_xmax = True, Nw = 31),
#    profile_class('realspace',q, dmax = 125, Nw = 101))

C1 = regals.component(
    regals.concentration_class('smooth', x, xmin = x.min(), xmax = x.max(), is_zero_at_xmin = False, 
                               is_zero_at_xmax = False, Nw = 31),
    regals.profile_class('realspace',newq, dmax = 160, Nw = 101))

C2 = regals.component(
    regals.concentration_class('smooth', x, xmin = x.min(), xmax = x.max(), is_zero_at_xmin = False, 
                               is_zero_at_xmax = False, Nw = 31),
    regals.profile_class('realspace',newq, dmax = 170, Nw = 101))

#C3 = regals.component(
#    regals.concentration_class('smooth', x, xmin = 0.1, xmax = 3.5, is_zero_at_xmin = False, is_zero_at_xmax = False, Nw = 31),
#    regals.profile_class('realspace',newq, dmax = 150, Nw = 101))

print(x)


## Step 6: Run REGALS

In [ ]:
M = regals.mixture([C1,C2])

R = regals.regals(iT,errT) # creat REGALS object
M.lambda_concentration = np.array([1E1,1E1,1E1])
M.lambda_profile = np.array([1E11,1E11,1E11])
# set stopFun to return true when 400 iterations is exceeded
stop_fun = lambda num_iter, params: [num_iter >= 400, 'max_iter']

update_fun = lambda num_iter, mix, params,resid: \
    print('%2d, x2 = %f, delta_profile = %s'%(num_iter,params['x2'],
                                              np.array2string(params['delta_profile'],precision=3))) if num_iter%10 == 0 else None

[M1,params,resid,exit_cond] = R.run(M,stop_fun,update_fun)   

## Step 7: Extract Data

In [ ]:
# P(r) functions
r1 = M1.components[0].profile.w
pr1 = np.concatenate([[0],M1.u_profile[0],[0]])

r2 = M1.components[1].profile.w
pr2 = np.concatenate([[0],M1.u_profile[1],[0]])


# Particle components (no regularization)
[I1,sigma1] = M1.extract_profile(iT,errT,0)
[I2,sigma2] = M1.extract_profile(iT,errT,1)


# Concentration components (no regularization)
p1 = M1.extract_concentration(iT,errT,0)[0]
p2 = M1.extract_concentration(iT,errT,1)[0]

# Regularized concentrations
x1 = M1.components[0].concentration.w
c1 = M1.u_concentration[0]
x2 = M1.components[1].concentration.w
c2 = M1.u_concentration[1]

c1.shape


## Step 8: Plot Data

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=[70,70])

#concentrations vs x
axs[0, 0].semilogx(10**x1, c1, linewidth=10, alpha=.8)
axs[0, 0].semilogx(10**x2, c2, linewidth=10, alpha=.8)
#axs[0, 0].semilogx(10**x3, c3, linewidth=10, alpha=0.8)

axs[0, 0].scatter(10**x, p1, s=500, zorder=1)
axs[0, 0].scatter(10**x, p2, s=500, zorder=1)
#axs[0, 0].scatter(10**x, p3, s=500, zorder=1)

axs[0, 0].set_xlabel('time ($\mu$s)', fontsize=70, fontweight='bold')
axs[0, 0].set_ylabel('Concentrations', fontsize=70, fontweight='bold')
axs[0, 0].set_title('Component Concentration', fontsize=80, fontweight='bold')
axs[0, 0].tick_params(labelsize=60)


#chi2 vs x
axs[1, 0].semilogx(10**x, np.mean(resid ** 2, 0), linewidth=10)
axs[1, 0].set_xlabel('time ($\mu$s)', fontsize=70, fontweight='bold')
axs[1, 0].set_ylabel('$\chi^2$',fontsize=70, fontweight='bold')
axs[1, 0].set_title('Fit Quality', fontsize=80, fontweight='bold')
axs[1, 0].tick_params(labelsize=60)

#P(r) vs r
axs[0, 1].plot(r1, pr1, linewidth=10)
axs[0, 1].plot(r2, pr2, linewidth=10)
#axs[0, 1].plot(r3, pr3, linewidth=10)

axs[0, 1].set_xlabel('$r (Å)$', fontsize=70, fontweight='bold')
axs[0, 1].set_ylabel('$P(r)$', fontsize=70, fontweight='bold')
axs[0, 1].set_title('Density Distribution', fontsize=80, fontweight='bold')
axs[0, 1].tick_params(labelsize=60)

#extracted profiles
axs[1, 1].plot(newq, I1, label='Component 1', linewidth=10)
#axs[1, 1].errorbar(newq, I1, sigma1, capsize=4, label='Component 1')
axs[1, 1].fill_between(newq, I1-sigma1, I1+sigma1, alpha=0.3)
#axs[1, 1].errorbar(newq, I2, sigma2, capsize=4, label='Component 2')
axs[1, 1].plot(newq, I2, label='Component 2', linewidth=10)
axs[1, 1].fill_between(newq, I2-sigma2, I2+sigma2, alpha=0.3)
#axs[1, 1].plot(newq, I3, label='Component 3', linewidth=10)
#axs[1, 1].fill_between(newq, I3-sigma3, I3+sigma3, alpha=0.3)
#axs[1, 1].errorbar(q, I3, sigma3, capsize=4, label='Component 3')
axs[1, 1].set_xscale('log')
axs[1, 1].set_yscale('linear')
axs[1, 1].set_xlabel('$q (Å^{-1})$', fontsize=70, fontweight='bold')
axs[1, 1].set_ylabel('Extracted profiles', fontsize=70, fontweight='bold')
axs[1, 1].set_title('REGALS Components', fontsize=80, fontweight='bold')
axs[1, 1].tick_params(labelsize=60)

legend_properties = {'weight':'bold', 'size':70}
plt.legend(loc='best', prop=legend_properties, ncol=1)

make_dir(f='./OUTPUT/TUTORIAL6/REGALS/')
plt.savefig('./OUTPUT/TUTORIAL6/REGALS/tutorial6_regals.png', 
                bbox_inches='tight')
plt.show()

# Iterating Kinetic Analysis

Because we run the kinetic analysis on a set of average, buffer-subtracted TR, T-Jump SAXS difference curves, we do not need to loop over multiple time delays to run the analysis. However, there may be instances where you would like to iterate over several data sets. This can be done by including the relevant functions inside a loop. If using this method it is recommended to use the `make_flist()` function to make the file list instead of manually indicating the files as done above. 


<div class="alert alert-block alert-success">
    
    <i class="fa fa-check-circle"></i>&nbsp; <b>Congratulations!</b><br>
    You completed the kinetic analysis tutorial. You are now ready to move on to the <a href="https://gitlab.oit.duke.edu/tr_t-jump_saxs/y22-23/-/blob/main/TUTORIALS/tutorial7_saxs_modeling.ipynb?ref_type=heads"> last tutorial</a>!  
    </div>